In [ ]:
#default_exp dataset

# Dataset Building

> Processes all included sources and compiles them into a single dataset file `data/dataset/dataset.zip`.

In [ ]:
#|export

from __future__ import annotations

from DataImporters.sources.core import Source, process
from DataImporters.annotation import load_annotations, apply_annotations

import os
import shutil
import pandas as pd
import pprint

In [ ]:
#export

class DatasetPaths:
    """Container for all the useful paths used to compile a `Dataset`."""
    def __init__(self, data_path: str, dataset_name: str, annotation_path: str = None):
        self.data_path = data_path
        self.dataset_name = dataset_name
        self.intermediate_path = os.path.join(data_path, "intermediate/")
        self.output_path = os.path.join(data_path, self.dataset_name)
        self.audio_output_path = os.path.join(self.output_path, "audio/")
        self.annotation_path = annotation_path
        self.metadata_output_path = os.path.join(self.output_path, "metadata.csv")
    
    def annotation(self, annotation_path: str) -> DatasetPaths:
        if not os.path.exists(annotation_path):
            raise FileNotFoundError(f"Annotation file {annotation_path} does not exist")
        
        self.annotation_path = annotation_path
        return self
    
    def has_annotation(self) -> bool:
        return self.annotation_path is not None
    
    def __repr__(self) -> str:
        return pprint.pformat(self.__dict__)

In [ ]:
#hide

paths = DatasetPaths(data_path="data", dataset_name="test")
assert paths.annotation_path is None
assert paths.output_path == "data/test"

try:
    paths = paths.annotation("nope.csv")
    raise Exception("Should raise FileNotFoundError if annotation file does not exist")
except FileNotFoundError:
    pass

paths

{'annotation_path': None,
 'audio_output_path': 'data/test/audio/',
 'data_path': 'data',
 'dataset_name': 'test',
 'intermediate_path': 'data/intermediate/',
 'metadata_output_path': 'data/test/metadata.csv',
 'output_path': 'data/test'}

In [ ]:
#exporti

def _sync_audio_files(source_dir: str, target_dir: str):
    """Synchronises audio files from source_dir to target_dir. Only copying new files."""
    for filename in os.listdir(source_dir):
        if filename.endswith(".wav"):
            target_file_path = os.path.join(target_dir, filename)
            if not os.path.exists(target_file_path):
                shutil.copy2(os.path.join(source_dir, filename), target_file_path)

def _delete_audio_files(filenames: str, target_dir: str):
    """Deletes all audio files in target_dir."""
    for file in filenames:
        os.remove(os.path.join(target_dir, file))

In [ ]:
#export

class Dataset:
    def __init__(
        self,
        sources: list[Source],
        paths: DatasetPaths
    ):
        self.sources = sources
        self.paths = paths

    def _prepare_output(self):
        if not os.path.exists(self.paths.audio_output_path):
            os.makedirs(self.paths.audio_output_path)

    def _compile_source(self, source: str):
        source_path = os.path.join(self.paths.intermediate_path, source.name)
        _sync_audio_files(source_path, self.paths.audio_output_path)
        return pd.read_csv(os.path.join(source_path, "metadata.csv"))

    def _apply_annotations(self, dataset_metadata: pd.DataFrame):
        if not self.paths.has_annotation():
            return dataset_metadata

        annotations = load_annotations(self.paths.annotation_path)
        # We could apply all these before copying the files, but that would spread the logic
        # across multiple locations and make it harder to understand. I have no performance
        # concerns at this point, so I'm leaving it like this.
        metadata = apply_annotations(annotations, dataset_metadata)
        _delete_audio_files(annotations.deletes["filename"], self.paths.audio_output_path)
        return metadata
    
    def _package_data(self):
        # It syncs with existing dataset, only zipping changed files
        os.system(
            f"cd {self.paths.data_path} ; zip -qq -FSr {self.paths.dataset_name}.zip {self.paths.dataset_name}/"
        )
    
    def compile(self) -> pd.DataFrame:
        """Compiles a dataset and returns the newly created metadata (already saved)."""
        self._prepare_output()

        dataset_metadata = pd.DataFrame()
        for source in self.sources:
            source_metadata = self._compile_source(source)
            dataset_metadata = pd.concat([dataset_metadata, source_metadata])
        
        clean_dataset = dataset_metadata.drop_duplicates("filename")
        diff = len(dataset_metadata) - len(clean_dataset)
        if diff != 0:
            print(f"Warning: {diff} duplicate rows found. Some rows were dropped (all files copied).")
            dataset_metadata = clean_dataset
        
        dataset_metadata = self._apply_annotations(dataset_metadata)
        dataset_metadata.to_csv(self.paths.metadata_output_path, index=False)

        self._package_data()

        return dataset_metadata

In [ ]:
#hide_input

from nbdev.showdoc import show_doc
show_doc(Dataset.compile)

<h4 id="Dataset.compile" class="doc_header"><code>Dataset.compile</code><a href="__main__.py#L25" class="source_link" style="float:right">[source]</a></h4>

> <code>Dataset.compile</code>()

Compiles a dataset and returns the newly created metadata (already saved).